In [14]:
from langchain import PromptTemplate
from langchain.tools import BaseTool
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import FAISS
import pickle
from langchain.callbacks.manager import AsyncCallbackManagerForToolRun, CallbackManagerForToolRun
from typing import Optional


llm = ChatOpenAI(temperature=0)

class LangchainDocSearch(BaseTool):
    name = "langchain_doc_search"
    description = "Langchain expert who will tell you how to do specific things using langchain."

    def load_vector_store(self):
        """
        Loads the langchain vector store from pickle into a local file
        """
        global vector_store
        with open("langchain_vectorstore.pkl", "rb") as f:
            vector_store = pickle.load(f)

    def _run(self, query: str) -> str:
        """
        Search the docs and return the most relevant pages content which we will use to feed to the model
        """
        res = vector_store.similarity_search(query)
        if res:
            return res[0].page_content, res[0].metadata['source']
    
    async def _arun(self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("langchain_doc_search does not support async")

In [15]:
lds = LangchainDocSearch()
lds.load_vector_store()

In [16]:
similar_docs, source = lds._run('Load a CSV document')

In [17]:
from pprint import pprint
pprint(similar_docs)
pprint(source)

('.ipynb\n'
 '.pdf\n'
 'CSV\n'
 ' Contents \n'
 'Customizing the csv parsing and loading\n'
 'Specify a column to identify the document source\n'
 'CSV#\n'
 'A comma-separated values (CSV) file is a delimited text file that uses a '
 'comma to separate values. Each line of the file is a data record. Each '
 'record consists of one or more fields, separated by commas.\n'
 'Load csv data with a single row per document.\n'
 'from langchain.document_loaders.csv_loader import CSVLoader\n'
 "loader = CSVLoader(file_path='./example_data/mlb_teams_2012.csv')\n"
 'data = loader.load()\n'
 'print(data)')
'rtdocs/langchain.readthedocs.io/en/latest/modules/indexes/document_loaders/examples/csv.html'
